In [1]:
from wordcloud import WordCloud
import imageio
import sys, os
import numpy as np
import requests
import json
from flask import Flask, jsonify, render_template, request, redirect, url_for
from flask import abort
from bs4 import BeautifulSoup


from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from operator import itemgetter
import pandas as pd
import matplotlib.pyplot as plt
import imageio
from wordcloud import WordCloud
from PIL import Image
import re, datetime
import shutil

In [94]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

ModuleNotFoundError: No module named 'pycontractions'

In [3]:
GENIUS_API_KEY = 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'
genius_base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer ' + GENIUS_API_KEY}

In [11]:
nrc_df = pd.read_csv("static/data/NRC-Emotion-Lexicon-v0.92.csv")

In [12]:
nrc_df

,Word,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,1,0,0,0,1,0,1,0,0
3,abandoned,0,1,1,0,0,1,0,1,0,0
4,abandonment,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,0,0,0,0,0,0,0,0,0,0
14180,zoology,0,0,0,0,0,0,0,0,0,0


In [2]:
def get_song_details(song_id):
    song_dict = {}

    # Get song metadata
    search_url = genius_base_url + '/songs/' + str(song_id)
    response = requests.get(search_url, headers=headers)
    song = response.json()
    song_dict['id'] = song['response']['song']['id']
    song_dict['title'] = song['response']['song']['title']
    song_dict['artist'] = song['response']['song']['primary_artist']['name']
    song_dict['artist_image'] = song['response']['song']['primary_artist']['image_url']
    song_dict['url'] = song['response']['song']['url']
    song_dict['album'] = song['response']['song']['album']['name']
    song_dict['year'] = song['response']['song']['release_date']
    song_dict['song_image'] = song['response']['song']['song_art_image_thumbnail_url']

    # Get song lyrics
    response = requests.get(song['response']['song']['url'], headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    lyrics = soup.find('div', class_='lyrics').get_text()
    lyrics = lyrics.replace('\n', ' ').replace('\r', ' ')
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    song_dict['lyrics'] = lyrics
    # print(song_dict)
    return song_dict

In [95]:
# Giving Credits to https://stackoverflow.com/questions/54396405/how-can-i-preprocess-nlp-text-lowercase-remove-special-characters-remove-numb
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]

    return filtered_words

In [4]:
song_data_dict = get_song_details(2336647)

In [100]:
lyrics = song_data_dict['lyrics']
lyrics = lyrics.lower()
lyrics

"   i will leave my heart at the door i won't say a word they've all been said before, you know so why don't we just play pretend like we're not scared of what is coming next or scared of having nothing left?   look, don't get me wrong i know there is no tomorrow all i ask is   if this is my last night with you hold me like i'm more than just a friend give me a memory i can use take me by the hand while we do what lovers do it matters how this ends 'cause what if i never love again?   i don't need your honesty it's already in your eyes and i'm sure my eyes, they speak for me no one knows me like you do and since you're the only one that mattered tell me, who do i run to?   look, don't get me wrong i know there is no tomorrow all i ask is   if this is my last night with you hold me like i'm more than just a friend give me a memory i can use take me by the hand while we do what lovers do it matters how this ends 'cause what if i never love again?   let this be our lesson in love let this

In [101]:
lyrics = preprocess(lyrics)

In [121]:
lyrics_df = pd.DataFrame(lyrics, columns=['Word'])

In [122]:
lyrics_df = lyrics_df.merge(nrc_df, how="inner", on="Word")
lyrics_df = lyrics_df.drop(columns=["Word"])

In [123]:
lyrics_df

,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,0,1,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0
9,0,1,0,0,0,0,0,0,0,0


In [124]:
results_df= pd.DataFrame(lyrics_df.sum())

In [125]:
results_df

,0
Positive,10
Negative,6
Anger,2
Anticipation,4
Disgust,2
Fear,1
Joy,7
Sadness,2
Surprise,1
Trust,6


In [105]:
lyrics_df

,0
0,leave
1,heart
2,door
3,say
4,word
...,...
98,matters
99,ends
100,cause
101,never


In [103]:
nrc_df

,Word,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,1,0,0,0,1,0,1,0,0
3,abandoned,0,1,1,0,0,1,0,1,0,0
4,abandonment,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,0,0,0,0,0,0,0,0,0,0
14180,zoology,0,0,0,0,0,0,0,0,0,0


In [99]:
print(lyrics)

['leave', 'heart', 'door', 'say', 'word', "they've", 'said', ',', 'know', 'play', 'pretend', 'like', "we're", 'scared', 'coming', 'next', 'scared', 'nothing', 'left', '?', 'look', ',', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', "i'm", 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', "'", 'cause', 'never', 'love', '?', 'need', 'honesty', 'already', 'eyes', "i'm", 'sure', 'eyes', ',', 'speak', 'one', 'knows', 'like', 'since', 'one', 'mattered', 'tell', ',', 'run', '?', 'look', ',', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', "i'm", 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', "'", 'cause', 'never', 'love', '?', 'let', 'lesson', 'love', 'let', 'way', 'remember', 'us', 'want', 'cruel', 'vicious', "ain't", 'asking', 'forgiveness', 'ask', 'last', 'night', 'hold', 'like', "i'm", 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', "'", '

In [80]:
a = preprocess(lyrics)
print(a)

['leave', 'heart', 'door', 'say', 'word', 'said', 'know', 'play', 'pretend', 'like', 'scared', 'coming', 'next', 'scared', 'nothing', 'left', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', 'cause', 'never', 'love', 'need', 'honesty', 'already', 'eyes', 'sure', 'eyes', 'speak', 'one', 'knows', 'like', 'since', 'one', 'mattered', 'tell', 'run', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', 'cause', 'never', 'love', 'let', 'lesson', 'love', 'let', 'way', 'remember', 'want', 'cruel', 'vicious', 'asking', 'forgiveness', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memory', 'use', 'take', 'hand', 'lovers', 'matters', 'ends', 'cause', 'never', 'love']


In [82]:
a = preprocess(lyrics)
print(a)

['leav', 'heart', 'door', 'say', 'word', 'said', 'know', 'play', 'pretend', 'like', 'scare', 'come', 'next', 'scare', 'noth', 'left', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love', 'need', 'honesti', 'alreadi', 'eye', 'sure', 'eye', 'speak', 'one', 'know', 'like', 'sinc', 'one', 'matter', 'tell', 'run', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love', 'let', 'lesson', 'love', 'let', 'way', 'rememb', 'want', 'cruel', 'viciou', 'ask', 'forgiv', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love']


In [84]:
a = preprocess(lyrics)
print(a)

['leav', 'heart', 'door', 'say', 'word', 'said', 'know', 'play', 'pretend', 'like', 'scare', 'come', 'next', 'scare', 'noth', 'left', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love', 'need', 'honesti', 'alreadi', 'eye', 'sure', 'eye', 'speak', 'one', 'know', 'like', 'sinc', 'one', 'matter', 'tell', 'run', 'look', 'get', 'wrong', 'know', 'tomorrow', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love', 'let', 'lesson', 'love', 'let', 'way', 'rememb', 'want', 'cruel', 'viciou', 'ask', 'forgiv', 'ask', 'last', 'night', 'hold', 'like', 'friend', 'give', 'memori', 'use', 'take', 'hand', 'lover', 'matter', 'end', 'caus', 'never', 'love']
